# This notebook

This notebook contains first bits and pieces of the yet to be developed model correlating climate/environmental factors with conflict occurrence.

## Import libraries and file with settings

In [1]:
import conflict_model

import pandas as pd
import geopandas as gpd
from configparser import RawConfigParser
import matplotlib.pyplot as plt
import numpy as np
import datetime
import rasterstats as rstats
import xarray as xr
import rasterio as rio
import os, sys

In [2]:
conflict_model.utils.show_versions()

Python version: 3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)]
conflict_model version: 0.0.1-beta
geopandas version: 0.7.0
xarray version: 0.15.1
rasterio version: 1.1.0
pandas version: 1.0.3
numpy version: 1.18.1
matplotlib version: 3.2.1
rasterstats version: 0.14.0


Geopandas versions lower than 0.7.0 do not yet have the clip function. The notebook will thus not work with these versions.

In [3]:
if gpd.__version__ < '0.7.0':
    sys.exit('please upgrade geopandas to version 0.7.0, your current version is {}'.format(gpd.__version__))

In this file all the settings for the analysis are defined. By 'parsing' it, all values are read for different sections. This is a simple way to make the code independent of the input data and settings.

In [4]:
settings_file = r'../data/run_setting.cfg'

In [5]:
config = RawConfigParser(allow_no_value=True)
config.read(settings_file)

['../data/run_setting.cfg']

In [6]:
#out_dir
out_dir = config.get('general','output_dir')
if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
print('for the record, saving output to folder {}'.format(out_dir) + os.linesep)

for the record, saving output to folder C:\Users\hoch0001\Documents\_code\conflict_model\data\OUT



# Applying functions

In [7]:
gdf = conflict_model.utils.get_geodataframe(config)

reading csv file to dataframe C:\Users\hoch0001\Documents\_code\conflict_model\data\UCDP/ged191.csv
...DONE

translating to geopandas dataframe
...DONE



In [8]:
conflict_gdf, extent_gdf = conflict_model.selection.select(gdf, config)

filtering on conflict properties...
...filtering key best with lower value 5
...filtering key type_of_violence with value 1
...passing key country as it is empty
focussing on period between 2000 and 2005

reading extent and spatial aggregation level from file C:\Users\hoch0001\Documents\_code\conflict_model\data\waterProvinces/waterProvinces_Africa.shp
...DONE

clipping datasets to extent
...DONE

clipping to climate zones['BWh', 'BSh']
...DONE



# Analysis per year

In a first step, we want to know in which countries there was conflict or not. TO that end, we first accumulate the number of fatalities per country and use this as proxy whether there was a conlfict or not (guess there is a rather strong like...).

In [9]:
sim_years = np.arange(config.getint('settings', 'y_start'), config.getint('settings', 'y_end'), 1)
sim_years

array([2000, 2001, 2002, 2003, 2004])

In [12]:
def rasterstats_GDP_PPP(gdf, config, sim_year, out_dir, saving_plots=False, showing_plots=False):
    
    print('calculating zonal statistics per aggregation level...')

    nc_fo = os.path.join(config.get('general', 'input_dir'), 
                         config.get('env_vars', 'GDP_PPP'))

    nc_ds = xr.open_dataset(nc_fo)

    nc_var = nc_ds['GDP_per_capita_PPP']

    years = nc_ds['time'].values
    years = years[years>=config.getint('settings', 'y_start')]
    years = years[years<=config.getint('settings', 'y_end')]

    affine = rio.open(nc_fo).transform

    gdf['zonal_stats_min'] = np.nan
    gdf['zonal_stats_max'] = np.nan
    gdf['zonal_stats_mean'] = np.nan

    nc_arr = nc_var.sel(time=sim_year)
    nc_arr_vals = nc_arr.values
    if nc_arr_vals.size == 0:
        raise ValueError('the data was found for this year in the nc-file {}, check if all is correct'.format(nc_fo))

    for i in range(len(gdf)):
        prov = gdf.iloc[i]
        zonal_stats = rstats.zonal_stats(prov.geometry, nc_arr_vals, affine=affine, stats="mean min max")
        gdf.loc[i, 'zonal_stats_min'] = zonal_stats[0]['min']
        gdf.loc[i, 'zonal_stats_max'] = zonal_stats[0]['max']
        gdf.loc[i, 'zonal_stats_mean'] = zonal_stats[0]['mean']
        
    print('...DONE' + os.linesep)

    fig, axes = plt.subplots(1, 3 , figsize=(20, 10))

    fig.suptitle(str(int(sim_year)), y=0.78)

    gdf.plot(ax=axes[0],
                column='zonal_stats_min',
                vmin=2000,
                vmax=15000,
                legend=True,
                legend_kwds={'label': "min GDP_PPP",
                                'orientation': "vertical",
                                'shrink': 0.5,
                                'extend': 'both'})

    gdf.plot(ax=axes[1],
                column='zonal_stats_max',
                vmin=2000,
                vmax=15000,
                legend=True,
                legend_kwds={'label': "max GDP_PPP",
                                'orientation': "vertical",
                                'shrink': 0.5,
                                'extend': 'both'})

    gdf.plot(ax=axes[2],
                column='zonal_stats_mean',
                vmin=2000,
                vmax=15000,
                legend=True,
                legend_kwds={'label': "mean GDP_PPP",
                                'orientation': "vertical",
                                'shrink': 0.5,
                                'extend': 'both'})

    plt.tight_layout()

    plt_name = 'GDP_PPP_zonal_stats_' + str(int(sim_year)) + '.png'
    plt_name = os.path.join(out_dir, plt_name)

    if saving_plots:
        plt.savefig(plt_name, dpi=300)

    if not showing_plots:
        plt.close()

    return gdf

In [13]:
for sim_year in np.arange(config.getint('settings', 'y_start'), config.getint('settings', 'y_end'), 1):

    print('entering year {}'.format(sim_year) + os.linesep)
    
    boolean_conflict_gdf = conflict_model.analysis.conflict_in_year_bool(conflict_gdf, 
                                                                         extent_gdf, 
                                                                         config, 
                                                                         sim_year, 
                                                                         out_dir, 
                                                                         showing_plots=False, saving_plots=True)
    
    GDP_PPP_gdf = rasterstats_GDP_PPP(boolean_conflict_gdf,
                                      config,
                                      sim_year,
                                      out_dir,
                                      saving_plots=True)

entering year 2000

determining whether a conflict took place or not...
...DONE

calculating zonal statistics per aggregation level...


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

entering year 2001

determining whether a conflict took place or not...
...DONE

calculating zonal statistics per aggregation level...


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

entering year 2002

determining whether a conflict took place or not...
...DONE

calculating zonal statistics per aggregation level...


...

...DONE

entering year 2003

determining whether a conflict took place or not...
...DONE

calculating zonal statistics per aggregation level...


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

entering year 2004

determining whether a conflict took place or not...
...DONE

calculating zonal statistics per aggregation level...


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

